# Preprocessing pipeline

Procesamos los escenarios de test

In [13]:
import pandas as pd
from pyspark.sql import SparkSession
from preprocess.enrichment import pipeline, meteo
from preprocess.cleaning import cleaner
import tensorflow as tf

RESULT_PATH = "../data/final_scenarios/answers_empty.csv"
DIR = "../data/final_scenarios/clean/"

# Inicializamos la sesión de Spark
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Cargamos el df de resultados
df_results = pd.read_csv(RESULT_PATH)

# Cargamos el modelo entrenado 
model = tf.keras.models.load_model(MODEL_PATH) 

# Cargamos los datos meteorológicos
meteo_dataset = meteo.MeteoDataset(METEO_PATH)
data_df = meteo_dataset.load_data('ruta/al/tu/archivo_de_datos.parquet')

# Creamos el pipeline de preprocesado
pipeline = pipeline.Pipeline()

# Creamos el depurador
depurador = depurador.Depurador()

for i in range(1, 194):
    file_name = f"scenario_{str(i).zfill(3)}.parquet"
    file_path = DIR + file_name
    # Leemos el dataframe con el escenario ya preprocesado con el formato del decoder
    df0 = spark.read.parquet(file_path)
    # Aplicamos el pipeline de procesamiento
    df1 = pipeline.apply_pipeline(df0, True)
    # Añadimos los datos meteorológicos
    df2 = meteo_dataset.merge(df1.toPandas())
    # Depuramos los datos
    df3 = depurador.apply_basic(df2) 
    # Aplicamos el pipeline de procesado especifico del modelo
    !! df4 = procesar para el modelo concreto(df3)
    # Sacamos las predicciones con el modelo
    predictions = model(df4)
    df4['prediction'] = predictions
    # Seleccionamos la fila con la predicción correspondiente para ese escenario
    # Sacamos la fila que nos interesa del df de resultados
    fila = df_results.iloc[i]

    df_validos = df4.join(fila, on=["ICAO", "HoldingPoint", "Runway"], how="inner")
    pred_mas_reciente = df_validos.loc[[df["Timestamp"].idxmax()]]

    # Rellenamos la información en el dataframe de resultados
    !! acordarse de poner -1 si no hay predicción
    df_results.iloc[i]["time_to_takeoff_s"] = pred_mas_reciente['prediction']

# Guardamos el dataframe de resultados
df_results.to_csv(RESULT_PATH, index=False) 


NameError: name 'MODEL_PATH' is not defined

# Predictions

Importamos el modelo final de la fase de entrenamiento

Realizamos las predicciones sobre el escenario procesado

Guardamos la predicción correspondiente en el CSV de respuestas

In [1]:
#----------------------------------------------
# Mis pruebecitas
#----------------------------------------------

In [1]:
# Lo hago primero con uno y luego hago el bucle
import pandas as pd

RESULT_PATH = "/Users/maria/Dropbox/UCM/PD2/despegues/src/data/final_scenarios/answers_empty.csv"
#MODEL_PATH = 
df_results = pd.read_csv(RESULT_PATH)

In [3]:
file_path = "../data/final_scenarios/clean/scenario_005.parquet"

from pyspark.sql import SparkSession 
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Creamos el dataframe con los datos del escenario post decoder
df0 = spark.read.parquet(file_path)
#df0.show(5)


25/05/08 00:37:56 WARN Utils: Your hostname, MacBook-Pro-de-Javi.local resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
25/05/08 00:37:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 00:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df0.printSchema()

root
 |-- Timestamp (date): string (nullable = true)
 |-- ICAO: string (nullable = true)
 |-- Downlink Format: long (nullable = true)
 |-- Typecode: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- TurbulenceCategory: string (nullable = true)
 |-- Callsign: string (nullable = true)
 |-- Speed: double (nullable = true)
 |-- Altitude (ft): double (nullable = true)
 |-- Flight status: string (nullable = true)



In [2]:
# Aplicamos el pipeline de procesamiento
from preprocess.enrichment import pipeline

p = pipeline.Pipeline()
df1 = p.apply_pipeline(df0, True)

df1.show(5)

2890


3
13
2
2


25/05/08 00:35:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------+--------------------+-------------+-----------------+------------------+----------+-------+--------------------+--------------------+--------------------+-----+-------------------------+---------------------+-------------------+
|  ICAO|Callsign|           Timestamp|Flight status|              lat|               lon|Designator| Runway|  TurbulenceCategory|first_on_ground_time|  first_holding_time|Speed|time_before_holding_point|time_at_holding_point|           time_10s|
+------+--------+--------------------+-------------+-----------------+------------------+----------+-------+--------------------+--------------------+--------------------+-----+-------------------------+---------------------+-------------------+
|344416| IBS1613|2025-03-04 12:31:...|    on-ground|40.49255629717293|-3.576112227006392|        Z4|36L/18R|Medium 2 (between...|2025-03-04 12:29:...|2025-03-04 12:31:...|  0.0|                    118.0|                  0.0|2025-03-04 12:31:10|
|344416| IBS1613

25/05/08 00:35:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-------------------+-----------------+-----------------+
|             Minute|last min takeoffs|last min landings|
+-------------------+-----------------+-----------------+
|2025-03-04 12:30:00|                0|                1|
|2025-03-04 12:32:00|                0|                1|
+-------------------+-----------------+-----------------+

+-------------------+------+--------+--------------------+-------------+-----------------+------------------+----------+-------+--------------------+--------------------+--------------------+-----+-------------------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-------------------+
|           time_10s|  ICAO|Callsign|           Timestamp|Flight status|              lat|               lon|Designator| Runway|  TurbulenceCategory|first_on_ground_time|  first_holding_time|Speed|time_before_holding_point|time

+-------+-------------------+-------------------+------+--------+--------------------+-------------+-----------------+------------------+----------+--------------------+--------------------+--------------------+-----+-------------------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-----------------+-----------------+--------------------+-------------------+----------+-----------------------------+----+-------+---------+
| Runway|             Minute|           time_10s|  ICAO|Callsign|           Timestamp|Flight status|              lat|               lon|Designator|  TurbulenceCategory|first_on_ground_time|  first_holding_time|Speed|time_before_holding_point|time_at_holding_point|   Z1|  KA6|  KA8|   K3|   K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|   LD|   LE|36R_18L|32R_14L|36L_18R|32L_14R|last min tak

+-------+-------------------+-------------------+------+--------+--------------------+-------------+-----------------+------------------+----------+--------------------+--------------------+--------------------+-----+-------------------------+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-----------------+-----------------+--------------------+-------------------+----------+-----------------------------+----+-------+---------+--------+
| Runway|             Minute|           time_10s|  ICAO|Callsign|           Timestamp|Flight status|              lat|               lon|Designator|  TurbulenceCategory|first_on_ground_time|  first_holding_time|Speed|time_before_holding_point|time_at_holding_point|   Z1|  KA6|  KA8|   K3|   K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|   LD|   LE|36R_18L|32R_14L|36L_18R|32L_14R|las

+--------------------+------+--------+-------------+-------+--------+--------------------+-----------------+------------------+-----------------+----------+-----------------+-------------------+-----------------------------+-------------------------+---------------------+----+-------+----------+--------------------+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
|           timestamp|  icao|callsign|holding_point| runway|operator| turbulence_category|              lat|               lon|last_min_takeoffs|last_event|last_min_landings|last_event_turb_cat|time_since_last_event_seconds|time_before_holding_point|time_at_holding_point|hour|weekday|is_holiday|     event_timestamp|  first_holding_time|first_on_ground_time|   Z1|  KA6|  KA8|   K3|   K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|   LD|   

+--------------------+------+--------+-------------+-------+--------+--------------------+-----------------+------------------+-----------------+----------+-----------------+-------------------+-----------------------------+-------------------------+---------------------+----+-------+----------+--------------------+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
|           timestamp|  icao|callsign|holding_point| runway|operator| turbulence_category|              lat|               lon|last_min_takeoffs|last_event|last_min_landings|last_event_turb_cat|time_since_last_event_seconds|time_before_holding_point|time_at_holding_point|hour|weekday|is_holiday|     event_timestamp|  first_holding_time|first_on_ground_time|   Z1|  KA6|  KA8|   K3|   K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|   LD|   

In [7]:
# Aplicamos el pipeline de procesamiento
from preprocess.enrichment import pipeline

for i in range(1, 194):
    file_path = f"../data/final_scenarios/clean/scenario_{i:03d}.parquet"
    df0 = spark.read.parquet(file_path)
    p = pipeline.Pipeline()
    df1 = p.apply_pipeline(df0)
    print("\n\n")
    print(i)
    df1.show()




1
+--------------------+------+--------+-------------+-------+--------+--------------------+------------------+-------------------+-----------------+----------+-----------------+--------------------+-----------------------------+-------------------------+---------------------+----+-------+----------+--------------------+--------------------+--------------------+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
|           timestamp|  icao|callsign|holding_point| runway|operator| turbulence_category|               lat|                lon|last_min_takeoffs|last_event|last_min_landings| last_event_turb_cat|time_since_last_event_seconds|time_before_holding_point|time_at_holding_point|hour|weekday|is_holiday|     event_timestamp|  first_holding_time|first_on_ground_time|   Z1|  KA6|  KA8|   K3|  K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|

+--------------------+------+--------+-------------+-------+--------+--------------------+-----------------+-------------------+-----------------+----------+-----------------+-------------------+-----------------------------+-------------------------+---------------------+----+-------+----------+--------------------+--------------------+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
|           timestamp|  icao|callsign|holding_point| runway|operator| turbulence_category|              lat|                lon|last_min_takeoffs|last_event|last_min_landings|last_event_turb_cat|time_since_last_event_seconds|time_before_holding_point|time_at_holding_point|hour|weekday|is_holiday|     event_timestamp|  first_holding_time|first_on_ground_time|   Z1|  KA6|  KA8|   K3|   K2|   K1|   Y1|   Y2|   Y3|   Y7|   Z6|   Z4|   Z2|   Z3|   LF|   L1|   LA|   LB|   LC|   LD| 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [4]:
df0.printSchema()


root
 |-- Timestamp (date): timestamp_ntz (nullable = true)
 |-- ICAO: string (nullable = true)
 |-- Downlink Format: long (nullable = true)
 |-- Typecode: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- TurbulenceCategory: string (nullable = true)
 |-- Callsign: string (nullable = true)
 |-- Speed: double (nullable = true)
 |-- Altitude (ft): double (nullable = true)
 |-- Flight status: string (nullable = true)



In [25]:
from pyspark.sql.functions import col

from pyspark.sql.functions import col, to_timestamp

# 1. Convertir a timestamp
df_ts = df0.withColumn("date_ts", to_timestamp(col("Timestamp (date)")))

# 2. Ordenar por timestamp descendente
df_sorted = df_ts.orderBy(col("date_ts").desc())

df_filtrado = df_sorted.filter((col("icao") == "344416"))
# 3. Seleccionar los dos últimos mensajes
df_last_two = df_filtrado.limit(3)


df_last_two.show()

+--------------------+------+---------------+--------+-----------------+------------------+------------------+--------+-----+-------------+-------------+--------------------+
|    Timestamp (date)|  ICAO|Downlink Format|Typecode|              lat|               lon|TurbulenceCategory|Callsign|Speed|Altitude (ft)|Flight status|             date_ts|
+--------------------+------+---------------+--------+-----------------+------------------+------------------+--------+-----+-------------+-------------+--------------------+
|2025-03-04 12:31:...|344416|             17|       7|40.49255553929809|-3.576034129003182|              NULL|    NULL| NULL|         NULL|    on-ground|2025-03-04 12:31:...|
|2025-03-04 12:31:...|344416|             17|       7|             NULL|              NULL|              NULL|    NULL|  0.0|         NULL|    on-ground|2025-03-04 12:31:...|
|2025-03-04 12:31:...|344416|             17|       7|40.49256793523239|-3.576065410267223|              NULL|    NULL|  3.0|

In [7]:
from pyspark.sql import functions as F
def getAirplaneCategories(df):
        """
        Genera un DataFrame con la categoría de cada aeronave.
        
        Parámetros:
            df: DataFrame de datos.
        Devuelve:
            DataFrame con las siguientes columnas: "ICAO", "TurbulenceCategory".
        """
        # Seleccionamos mensajes ADS-B
        df_filtered = df.filter(F.col("Downlink Format").isin([17, 18]))
        # Nos quedamos con los ICAOs y su tipo de avión
        df_filtered = df_filtered.filter(F.col("TurbulenceCategory").isNotNull() & (F.col("TurbulenceCategory") != "None") & (~F.isnan(F.col("TurbulenceCategory"))))
        df_result = df_filtered.select("ICAO", "TurbulenceCategory").dropDuplicates()
        return df_result

In [8]:
df2 = getAirplaneCategories(df0)
df2.show(5)

+------+--------------------+
|  ICAO|  TurbulenceCategory|
+------+--------------------+
|ab7376|Heavy (larger tha...|
|344099|Medium 2 (between...|
|3420cc|Medium 2 (between...|
|346303|Medium 2 (between...|
|39ceb0|Medium 2 (between...|
+------+--------------------+
only showing top 5 rows

